In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# load traffic and weather datasets
data = pd.read_csv('traffic.csv',index_col='Date',parse_dates=True)
daily_data = data.resample('d').sum()[['Fremont Bridge Total']]
weather_data = pd.read_csv('weather_Seattle.csv', index_col='DATE',parse_dates=True)
# fix TAVG column
weather_data.TAVG.fillna(0.5*(weather_data.TMAX+weather_data.TMIN), inplace=True)

# add day_of_week, month, year and covid_pandemic columns
daily_data['day_of_week'] = daily_data.index.dayofweek
daily_data['month'] = daily_data.index.month
daily_data['year'] = daily_data.index.year
daily_data['covid_pandemic'] = (daily_data.index>'2020-03').astype(int)

# add holiday column
from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()
holidays = cal.holidays('2012-10-03','2021-08-31')
daily_data['holiday'] = (pd.Series(1, index=holidays,name='holiday'))
daily_data['holiday'].fillna(0, inplace=True)

# add hours_of_daylight column
def hours_of_daylight(date, axis=23.44, latitude=47.61):
    days = (date-pd.to_datetime('2000-12-21')).days
    m = (1. - np.tan(np.radians(latitude)) * np.tan(np.radians(axis) * np.cos(days*2*np.pi/365.25)))
    return 24.*np.degrees(np.arccos(1-np.clip(m,0,2,))) / 180
daily_data['hours_daylight'] = daily_data.index.map(hours_of_daylight)

# add temperature, precipitation, average wind speed, and snowfall columns
daily_data['TAVG'] = weather_data.TAVG
daily_data['PRCP'] = weather_data.PRCP
daily_data['AWND'] = weather_data.AWND
daily_data['SNOW'] = weather_data.SNOW

daily_data.head()

FileNotFoundError: [Errno 2] File traffic.csv does not exist: 'traffic.csv'